In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
from bulletin import default_input, default_output, tables_path, root
default_output = join(default_output,'vacinacao')

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

hoje = pd.to_datetime(date.today())
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])

In [ ]:
situacao_labels = { 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
fator_risco_labels = { 0:'Sem Fator de Risco', 1:'Com Fator de Risco' }
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}
evolucao_labels = {1: 'Recuperados', 2: 'Óbitos', 3: 'Ativos'}

In [ ]:
vac = eSUS('vacinados')
vac.load()
vac.df.shape

In [ ]:
vac.df.groupby(['situacao_atual'])[['cns']].count().rename(index=situacao_labels, level='situacao_atual')

In [ ]:
cc = CasosConfirmados()

try:
    cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
except:
    cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}", compress=True)

cc.df.pivot_table(index='evolucao',values='id_notifica',aggfunc='count',margins=True,margins_name='Casos').rename(index={1: 'Recuperados',2: 'Óbitos', 3:'Ativos'},columns={'id_notifica':'qtde'}).T[['Casos','Óbitos','Recuperados','Ativos']]

In [ ]:
casos_confirmados = cc.df.rename(columns={'id_notifica':'id'})
casos_confirmados['faixa_etaria'] = np.digitize(casos_confirmados['idade'],tables['faixa_etaria_populacao']['bins'],right=False)

casos_confirmados['ano_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos_confirmados['mes_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')

casos_confirmados.groupby(['ano_caso','mes_caso','faixa_etaria'])[['id']].count().unstack([-1]).fillna(0).astype(int)

In [ ]:
vacinados = vac.df

In [ ]:
casos_vacinados_cns = pd.merge(vacinados.loc[vacinados['cns'].notna()],casos_confirmados[['cns','id','data_nascimento','nome_mae','paciente','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito']], on='cns', how='inner',suffixes=['_esus','_notifica'])
print(len(casos_vacinados_cns))

In [ ]:
casos_vacinados_cpf = pd.merge(vacinados.loc[vacinados['cpf'].notna()],casos_confirmados[['cpf','id','data_nascimento','nome_mae','paciente','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito']], on='cpf', how='inner',suffixes=['_esus','_notifica'])
print(len(casos_vacinados_cpf))

In [ ]:
casos_vacinados_mae = pd.merge(vacinados.loc[vacinados['hash_mae'].notna()],casos_confirmados[['hash_mae','id','data_nascimento','nome_mae','paciente','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito']], on='hash_mae', how='inner',suffixes=['_esus','_notifica'])
print(len(casos_vacinados_mae))

In [ ]:
casos_vacinados_nasc = pd.merge(vacinados.loc[vacinados['hash_nasc'].notna()],casos_confirmados[['hash_nasc','id','data_nascimento','nome_mae','paciente','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito']], on='hash_nasc', how='inner',suffixes=['_esus','_notifica'])
print(len(casos_vacinados_nasc))

In [ ]:
casos_vacinados = pd.concat([casos_vacinados_cns,casos_vacinados_cpf,casos_vacinados_mae,casos_vacinados_nasc], ignore_index=True).drop_duplicates('id', keep='first')
print(len(casos_vacinados))

In [ ]:
diffs = casos_vacinados.loc[~(
    (casos_vacinados['paciente_esus']==casos_vacinados['paciente_notifica']) |
    (casos_vacinados['data_nascimento_esus']==casos_vacinados['data_nascimento_notifica']) |
    (casos_vacinados['nome_mae_esus']==casos_vacinados['nome_mae_notifica']) 
), ['id','paciente_esus','paciente_notifica','data_nascimento_esus','data_nascimento_notifica','nome_mae_esus','nome_mae_notifica']]

diffs.loc[diffs['paciente_esus'].notna() & diffs['paciente_notifica'].notna(),'paciente_diff'] = diffs.loc[diffs['paciente_esus'].notna() & diffs['paciente_notifica'].notna(),['paciente_esus','paciente_notifica']].apply(lambda row: similar(row['paciente_esus'],row['paciente_notifica']),axis=1)

diffs.loc[diffs['nome_mae_esus'].notna() & diffs['nome_mae_notifica'].notna(),'nome_mae_diff'] = diffs.loc[diffs['nome_mae_esus'].notna() & diffs['nome_mae_notifica'].notna(),['nome_mae_esus','nome_mae_notifica']].apply(lambda row: similar(row['nome_mae_esus'],row['nome_mae_notifica']),axis=1)

diffs.loc[diffs['data_nascimento_esus'].notna() & diffs['data_nascimento_notifica'].notna(),'data_nascimento_diff'] = diffs.loc[diffs['data_nascimento_esus'].notna() & diffs['data_nascimento_notifica'].notna(),['data_nascimento_esus','data_nascimento_notifica']].apply(lambda row: similar(row['data_nascimento_esus'].strftime('%d%m%Y'),row['data_nascimento_notifica'].strftime('%d%m%Y')),axis=1)

diffs['media_diff'] = diffs[[col for col in diffs.columns if 'diff' in col]].mean(axis=1)
diffs[[col for col in diffs.columns if 'diff' in col]]
diffs.loc[diffs['media_diff']<0.7]

In [ ]:
casos_vacinados = casos_vacinados.drop(index=diffs.loc[diffs['media_diff']<0.7].index)

In [ ]:
casos_vacinados = casos_vacinados.drop(columns=casos_vacinados[[col for col in casos_vacinados.columns if 'esus' in col or 'notifica' in col]].columns)

In [ ]:
del vac.df
del vacinados

In [ ]:
casos_vacinados.columns

In [ ]:
casos_vacinados.groupby(['situacao_atual','doses_aplicadas'])[['id']].count()

In [ ]:
casos_vacinados['situacao_contagio'] = 0
casos_vacinados.groupby('situacao_contagio')[['id']].count()

In [ ]:
casos_vacinados.loc[
    (
        (casos_vacinados['doses_aplicadas']=='1ª Dose') 
        &
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'])
    ) 
    |
    (
        (casos_vacinados['doses_aplicadas']=='1ª Dose,2ª Dose') 
        &
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'])
        &
        (casos_vacinados['data_diagnostico'] <= casos_vacinados['data_imunizacao']  )
    ) 
    | 
    (
        (casos_vacinados['doses_aplicadas']=='Única') 
        & 
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_dose_unica'])
        &
        (casos_vacinados['data_diagnostico'] <= casos_vacinados['data_imunizacao']  )
    )
, 'situacao_contagio'] = 1

casos_vacinados.groupby('situacao_contagio',dropna=False)[['id']].count()

In [ ]:
casos_vacinados.loc[
    casos_vacinados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose',
            '1ª Dose,2ª Dose,Adicional',
            'Única',
            'Única,Adicional',
        ]
    ) 
    &
    (casos_vacinados['data_diagnostico'] > casos_vacinados['data_imunizacao']  )
    
, 'situacao_contagio'] = 2

casos_vacinados.groupby('situacao_contagio',dropna=False)[['id']].count()

In [ ]:
casos_vacinados.loc[
    casos_vacinados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (casos_vacinados['data_diagnostico'] > casos_vacinados['data_imunizacao'])
    &
    (casos_vacinados['data_diagnostico'] <= casos_vacinados['data_imunizacao_reforco'])

, 'situacao_contagio'] = 2

casos_vacinados.groupby('situacao_contagio',dropna=False)[['id']].count()

In [ ]:
casos_vacinados.loc[
    casos_vacinados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (casos_vacinados['data_diagnostico'] > casos_vacinados['data_imunizacao_reforco'])

, 'situacao_contagio'] = 3

casos_vacinados.groupby('situacao_contagio',dropna=False)[['id']].count()

In [ ]:
casos_vacinados.groupby(['situacao_contagio','situacao_atual'])[['id']].count()

In [ ]:
#### selecionar colunas
casos = pd.merge(casos_confirmados,casos_vacinados[['id'] + [col for col in casos_vacinados.columns if not (col in casos_confirmados.columns)]],how='left',on='id')
casos['situacao_contagio'] = casos['situacao_contagio'].fillna(0)
casos.columns

In [ ]:
fator_risco_cols = cc.schema.loc[cc.schema['group_name']=='fator_risco','column'].values.tolist()
del cc.df
del casos_confirmados
del casos_vacinados

In [ ]:
casos.pivot_table(index='situacao_contagio',columns='evolucao', values='id', aggfunc='count', margins=True, margins_name='Total').rename(index=situacao_labels,level='situacao_contagio').rename(columns={1:'Recuperados', 2:'Óbitos', 3:'Ativos','Total':'Casos'})[['Casos','Óbitos','Recuperados','Ativos']]

In [ ]:
casos['fx'] = '60+'
casos.loc[(casos['idade']>=12)&(casos['idade']<60),'fx'] = '12-59'
casos.loc[(casos['idade']<12),'fx'] = '0-12'

In [ ]:
casos[fator_risco_cols]

In [ ]:
fator_risco_table = casos[['id'] + fator_risco_cols]
fator_risco_table.set_index('id',inplace=True)
del fator_risco_table['outras_morbidades']
del fator_risco_table['periodo_gestacao']

In [ ]:
fator_risco = fator_risco_table.transform(lambda x: x == 1)
fator_risco = fator_risco.sum(axis=1)
fator_risco.name = 'fator_risco'
fator_risco = fator_risco.transform(lambda x: 1 if x >= 1 else 0)

In [ ]:
casos = casos.join(fator_risco, on='id')
casos.loc[casos['fator_risco'].isna(),fator_risco_table.columns] = 3
casos.loc[casos['fator_risco']==3,'fator_risco'] = 0
casos['fator_risco'] = casos['fator_risco'].astype(int)

In [ ]:
total = casos.groupby(['situacao_contagio','evolucao'])[['id']].count().unstack(-1).fillna(0).astype(int).droplevel(0,1)
total = total.rename(index=situacao_labels, level='situacao_contagio').rename(columns=evolucao_labels, level='evolucao')
total['Casos'] = total.sum(1)
total.loc['Total',:] = total.sum(0)
total = total.astype(int)
total = total[['Casos','Óbitos','Recuperados','Ativos']]
total

In [ ]:
casos['ano_caso'] = casos['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')

In [ ]:
cc.tables['raca_cor'].set_index('id')['raca_cor'].to_dict()

In [ ]:
casos.loc[casos['raca_cor'].isin([0,6]),'raca_cor'] = 99

In [ ]:
fatores = [ f for f in fator_risco_cols if f not in ['outras_morbidades','periodo_gestacao','gestante_alto_risco','gestante']]

casos_fator_risco = casos.loc[(casos['fator_risco']==1)]

casos_fator_risco_situacao_contagio = {}

for situacao_contagio in sorted(casos_fator_risco['situacao_contagio'].unique()):
    fator_risco = casos_fator_risco.loc[casos_fator_risco['situacao_contagio']==situacao_contagio,fatores]
    n = len(fator_risco)
    fator_risco = fator_risco == 1
    fator_risco = fator_risco.sum(axis=0).to_frame(name='Quantidade')
    fator_risco['%'] = fator_risco['Quantidade'] / n
    # fator_risco['%'] = fator_risco['Quantidade'] / fator_risco['Quantidade'].sum()
    fator_risco = fator_risco.sort_values('%',ascending=False)
    fator_risco.loc['TOTAL DE ÓBITOS','Quantidade'] = n
    casos_fator_risco_situacao_contagio[situacao_contagio] = fator_risco
    
casos_fator_risco_situacao_contagio = pd.concat(casos_fator_risco_situacao_contagio,names=['situacao_contagio','fator_risco'],axis=1)
casos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'%')] = (casos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')] / casos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')].sum()).values
casos_fator_risco_situacao_contagio = casos_fator_risco_situacao_contagio.rename(columns=situacao_labels, level='situacao_contagio')
casos_fator_risco_situacao_contagio_csv = casos_fator_risco_situacao_contagio.set_axis([f"{l2} {l1}" for l1,l2 in casos_fator_risco_situacao_contagio.columns], axis=1)
# casos_fator_risco_situacao_contagio_csv.columns = ['Não Vacinados', '% Não Vacinados', 'Com Esquema Primário Incompleto', '% Esquema Primário Incompleto', 'Com Esquema Primário Completo', '% Esquema Primário Completo', 'Com Esquema Primário Completo + Reforço', '% Esquema Primário Completo + Reforço']
# casos_fator_risco_situacao_contagio_csv.to_csv(join(default_output,'casos_fator_risco_situacao_contagio.csv'), sep = ';', decimal=',', float_format='%.3f')
casos_fator_risco_situacao_contagio_csv

In [ ]:
cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.df = casos
cc_vacinados.save(replace=True)